# Data Loading and Preprocessing
Load HAM10000 dataset, implement data augmentation, and prepare few-shot episodes with support and query sets.

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
metadata = pd.read_csv('../input/ham10000/HAM10000_metadata.csv')
image_path = '../input/ham10000/'

# Function to load images
def load_images(df, image_path):
    images = []
    for img_id in df['image_id']:
        img = cv2.imread(os.path.join(image_path, f'{img_id}.jpg'))
        img = cv2.resize(img, (128, 128))
        images.append(img)
    return np.array(images)

# Load images
images = load_images(metadata, image_path)
labels = metadata['dx'].values

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Prepare few-shot episodes
def create_few_shot_episodes(images, labels, n_way=5, k_shot=1, k_query=1):
    unique_labels = np.unique(labels)
    episodes = []
    for _ in range(len(images) // (n_way * (k_shot + k_query))):
        selected_labels = np.random.choice(unique_labels, n_way, replace=False)
        support_set = []
        query_set = []
        for label in selected_labels:
            label_indices = np.where(labels == label)[0]
            selected_indices = np.random.choice(label_indices, k_shot + k_query, replace=False)
            support_set.append(images[selected_indices[:k_shot]])
            query_set.append(images[selected_indices[k_shot:]])
        episodes.append((np.array(support_set), np.array(query_set)))
    return episodes

# Create few-shot episodes
few_shot_episodes = create_few_shot_episodes(train_images, train_labels)

# Define UNet Architecture with Attention
Implement UNet with skip connections and self-attention mechanism for better feature extraction.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input, Activation, BatchNormalization, Add, Multiply
from tensorflow.keras.models import Model

# Attention block
def attention_block(x, g, inter_channel):
    theta_x = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(g)
    f = Activation('relu')(Add()([theta_x, phi_g]))
    psi_f = Conv2D(1, (1, 1), strides=(1, 1), padding='same')(f)
    rate = Activation('sigmoid')(psi_f)
    att_x = Multiply()([x, rate])
    return att_x

# UNet with Attention
def unet_with_attention(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    
    # Decoder
    up6 = UpSampling2D(size=(2, 2))(conv5)
    att6 = attention_block(conv4, up6, 512)
    merge6 = Concatenate()([att6, up6])
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = UpSampling2D(size=(2, 2))(conv6)
    att7 = attention_block(conv3, up7, 256)
    merge7 = Concatenate()([att7, up7])
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = UpSampling2D(size=(2, 2))(conv7)
    att8 = attention_block(conv2, up8, 128)
    merge8 = Concatenate()([att8, up8])
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = UpSampling2D(size=(2, 2))(conv8)
    att9 = attention_block(conv1, up9, 64)
    merge9 = Concatenate()([att9, up9])
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    
    model = Model(inputs, conv10)
    
    return model

# Create the model
unet_model = unet_with_attention()
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
unet_model.summary()

# Implement Few-Shot Learning Components
Create prototypical network components for few-shot learning, including support set embedding and query set comparison.

In [ ]:
# Implement Few-Shot Learning Components

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Prototypical Network Components
class PrototypicalNetwork(Model):
    def __init__(self, encoder):
        super(PrototypicalNetwork, self).__init__()
        self.encoder = encoder

    def call(self, support_set, query_set):
        # Embed the support set
        support_embeddings = self.encoder(support_set)
        support_embeddings = tf.reduce_mean(support_embeddings, axis=1)  # Average embeddings for each class

        # Embed the query set
        query_embeddings = self.encoder(query_set)

        return support_embeddings, query_embeddings

# Encoder Model
def create_encoder(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    model = Model(inputs, x)
    return model

# Create encoder and prototypical network
encoder = create_encoder()
proto_net = PrototypicalNetwork(encoder)

# Example usage with few-shot episodes
support_set, query_set = few_shot_episodes[0]
support_embeddings, query_embeddings = proto_net(support_set, query_set)

# Calculate distances between support and query embeddings
def euclidean_distance(a, b):
    return tf.sqrt(tf.reduce_sum(tf.square(a - b), axis=-1))

distances = euclidean_distance(tf.expand_dims(query_embeddings, 1), tf.expand_dims(support_embeddings, 0))

# Predict class based on nearest prototype
predictions = tf.argmin(distances, axis=-1)

# Print predictions
print(predictions)

# FASTGAN Implementation
Implement FASTGAN for data augmentation to enhance few-shot learning performance.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, ReLU, Flatten, Dense, Reshape
from tensorflow.keras.models import Model

# Define the generator model for FASTGAN
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(Dense(8 * 8 * 256, input_dim=latent_dim))
    model.add(Reshape((8, 8, 256)))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2D(3, kernel_size=3, padding='same', activation='tanh'))
    return model

# Define the discriminator model for FASTGAN
def build_discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Define the GAN model combining generator and discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

# Set parameters
latent_dim = 100
input_shape = (128, 128, 3)

# Build and compile the discriminator
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Build and compile the GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Function to generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow((predictions[i] * 127.5 + 127.5).astype(np.uint8))
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Training the GAN
def train_gan(generator, discriminator, gan, dataset, latent_dim, epochs=10000, batch_size=64, save_interval=200):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, dataset.shape[0], half_batch)
        real_images = dataset[idx]
        real_labels = np.ones((half_batch, 1))

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")
            generate_and_save_images(generator, epoch, np.random.normal(0, 1, (16, latent_dim)))

# Prepare the dataset for GAN training
train_images = (train_images.astype(np.float32) - 127.5) / 127.5

# Train the GAN
train_gan(generator, discriminator, gan, train_images, latent_dim)

# Model Training
Train the model using episodic training paradigm, combining few-shot learning with GAN-augmented data.

In [ ]:
# Model Training

# Define the episodic training function
def train_prototypical_network(proto_net, episodes, epochs=10):
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    for epoch in range(epochs):
        epoch_loss = 0
        for support_set, query_set in episodes:
            with tf.GradientTape() as tape:
                support_embeddings, query_embeddings = proto_net(support_set, query_set)
                distances = euclidean_distance(tf.expand_dims(query_embeddings, 1), tf.expand_dims(support_embeddings, 0))
                predictions = tf.argmin(distances, axis=-1)
                loss = loss_fn(tf.range(len(predictions)), predictions)
            gradients = tape.gradient(loss, proto_net.trainable_variables)
            optimizer.apply_gradients(zip(gradients, proto_net.trainable_variables))
            epoch_loss += loss

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(episodes)}')

# Train the Prototypical Network
train_prototypical_network(proto_net, few_shot_episodes)

# Integrate GAN-augmented data into training
def augment_with_gan(generator, support_set, k_shot, latent_dim):
    noise = np.random.normal(0, 1, (k_shot, latent_dim))
    generated_images = generator.predict(noise)
    return np.concatenate([support_set, generated_images], axis=0)

# Augment support sets with GAN-generated images
augmented_episodes = []
for support_set, query_set in few_shot_episodes:
    augmented_support_set = augment_with_gan(generator, support_set, k_shot=1, latent_dim=latent_dim)
    augmented_episodes.append((augmented_support_set, query_set))

# Train the Prototypical Network with augmented data
train_prototypical_network(proto_net, augmented_episodes)

# Explainability using saliency maps
def compute_saliency_maps(model, images, labels):
    images = tf.convert_to_tensor(images)
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
    gradients = tape.gradient(loss, images)
    saliency_maps = tf.reduce_max(tf.abs(gradients), axis=-1)
    return saliency_maps

# Compute saliency maps for validation images
val_images_tensor = tf.convert_to_tensor(val_images)
val_labels_tensor = tf.convert_to_tensor(val_labels)
saliency_maps = compute_saliency_maps(proto_net.encoder, val_images_tensor, val_labels_tensor)

# Display saliency maps
def display_saliency_maps(images, saliency_maps):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(images[0].astype(np.uint8))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    axes[1].imshow(saliency_maps[0], cmap='hot')
    axes[1].set_title('Saliency Map')
    axes[1].axis('off')
    plt.show()

# Display saliency map for a sample validation image
display_saliency_maps(val_images, saliency_maps)

# Saliency Maps and Explainability
Generate saliency maps using gradient-based methods to visualize model decisions.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Explainability using saliency maps
def compute_saliency_maps(model, images, labels):
    images = tf.convert_to_tensor(images, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
    gradients = tape.gradient(loss, images)
    saliency_maps = tf.reduce_max(tf.abs(gradients), axis=-1)
    return saliency_maps

# Compute saliency maps for validation images
val_images_tensor = tf.convert_to_tensor(val_images, dtype=tf.float32)
val_labels_tensor = tf.convert_to_tensor(val_labels, dtype=tf.int64)
saliency_maps = compute_saliency_maps(proto_net.encoder, val_images_tensor, val_labels_tensor)

# Display saliency maps
def display_saliency_maps(images, saliency_maps):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(images[0].astype(np.uint8))
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    axes[1].imshow(saliency_maps[0], cmap='hot')
    axes[1].set_title('Saliency Map')
    axes[1].axis('off')
    plt.show()

# Display saliency map for a sample validation image
display_saliency_maps(val_images, saliency_maps)